# XGBoost Base Model

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import mlflow
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from xgboost import XGBClassifier
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
# from hyperopt.pyll import scope


warnings.filterwarnings("ignore")

# Data Setup

In [3]:
# read input data
churn = pd.read_csv("../data/churn.txt")
pd.set_option("display.max_columns", 500)

NameError: name 'pd' is not defined

In [2]:
churn = churn.drop(["Phone", "Day Charge", "Eve Charge", "Night Charge", "Intl Charge"], axis=1)
churn["Area Code"] = churn["Area Code"].astype(object)

NameError: name 'churn' is not defined

# Train Model

In [4]:
# Extract feature and target arrays
X, y = churn.drop('Churn?', axis=1), churn[['Churn?']]

In [5]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

In [6]:
y['Churn?'] = y['Churn?'].replace({'True.': 1, 'False.': 0})

In [7]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [35]:
X_test.shape

(1250, 15)

In [36]:
X_test.head()

,State,Account Length,Area Code,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,Intl Mins,Intl Calls,CustServ Calls
2764,FL,131,707,no,yes,400,4.629849,4,3.501916,1,4.098925,150,4.328442,4,6
4767,KS,42,758,yes,yes,500,6.181480,3,2.979401,1,2.797090,150,6.228276,7,7
3814,GA,187,716,no,no,0,7.886689,3,6.175369,4,6.698013,400,4.794217,7,7
3499,MI,136,777,yes,no,0,5.657278,2,4.398823,7,8.741026,300,5.569440,9,1
2735,NH,200,787,yes,yes,400,2.797435,4,4.575284,3,3.373482,200,4.977393,3,9


In [39]:
X_test_customer = X_test.iloc[0]
y_test_customer = y_test.iloc[0]

In [40]:
print(X_test_customer)
print(y_test_customer)

State                   FL
Account Length         131
Area Code              707
Int'l Plan              no
VMail Plan             yes
VMail Message          400
Day Mins          4.629849
Day Calls                4
Eve Mins          3.501916
Eve Calls                1
Night Mins        4.098925
Night Calls            150
Intl Mins         4.328442
Intl Calls               4
CustServ Calls           6
Name: 2764, dtype: object
Churn?    0
Name: 2764, dtype: int64


In [41]:
import pandas as pd

# Sample object to check
my_object = X_test

# Check if the object is a DataFrame
if isinstance(my_object, pd.DataFrame):
    print("It's a DataFrame!")
else:
    print("It's not a DataFrame.")


It's a DataFrame!


In [29]:
import pandas as pd

# Sample object to check
my_object = X_test_customer

# Check if the object is a pandas data structure (DataFrame or Series)
if issubclass(type(my_object), (pd.DataFrame, pd.Series)):
    print("It's a pandas data structure!")
else:
    print("It's not a pandas data structure.")


It's a pandas data structure!


In [31]:
import pandas as pd

# Sample pandas object (DataFrame or Series)
my_object = X_test_customer

# Check the type of the object
obj_type = type(my_object)

# Print the type
print(obj_type)


<class 'pandas.core.frame.DataFrame'>


In [42]:
X_test_customer.dtypes

dtype('O')

In [34]:
X_test_customer.shape

(1, 15)

In [43]:
customer = X_test_customer.to_json()
print(customer)

{"State":"FL","Account Length":131,"Area Code":707,"Int'l Plan":"no","VMail Plan":"yes","VMail Message":400,"Day Mins":4.6298486156,"Day Calls":4,"Eve Mins":3.5019160814,"Eve Calls":1,"Night Mins":4.0989253823,"Night Calls":150,"Intl Mins":4.3284419486,"Intl Calls":4,"CustServ Calls":6}


In [63]:
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test_customer, label=y_test_customer, enable_categorical=True)

# Train the best model

In [64]:
best_params = {
    'learning_rate'	: 0.2611886716276454,
    'max_depth' : 39,
    'min_child_weight' : 4.490391995734931,
    'objective' : 'binary:logistic',
    'reg_alpha' : 0.044567672488398144,
    'reg_lambda' : 0.11968534468462336,
    'seed' : 42
}

booster = xgb.train(
    params=best_params,
    dtrain=dtrain,
    num_boost_round=1000,
    evals=[(dtest,"test")],
    early_stopping_rounds=50
)

[0]	test-logloss:0.74476
[1]	test-logloss:0.76976
[2]	test-logloss:0.58331


[3]	test-logloss:0.65161
[4]	test-logloss:0.60024
[5]	test-logloss:0.71124
[6]	test-logloss:0.62346
[7]	test-logloss:0.76900
[8]	test-logloss:0.86313
[9]	test-logloss:0.95938
[10]	test-logloss:0.82591
[11]	test-logloss:0.82591
[12]	test-logloss:0.77312
[13]	test-logloss:0.79365
[14]	test-logloss:0.80702
[15]	test-logloss:0.78836
[16]	test-logloss:0.76066
[17]	test-logloss:0.75369
[18]	test-logloss:0.74427
[19]	test-logloss:0.89402
[20]	test-logloss:0.83280
[21]	test-logloss:0.82879
[22]	test-logloss:0.95955
[23]	test-logloss:0.98658
[24]	test-logloss:0.92268
[25]	test-logloss:0.83878
[26]	test-logloss:0.77443
[27]	test-logloss:0.87814
[28]	test-logloss:0.81302
[29]	test-logloss:0.79382
[30]	test-logloss:0.89017
[31]	test-logloss:0.93130
[32]	test-logloss:0.96032
[33]	test-logloss:0.96375
[34]	test-logloss:0.96375
[35]	test-logloss:1.07160
[36]	test-logloss:1.10371
[37]	test-logloss:1.01809
[38]	test-logloss:0.95089
[39]	test-logloss:0.96096
[40]	test-logloss:1.07586
[41]	test-logloss:1

# Make Single Prediction

In [67]:
# make prediction
y_pred_prob = booster.predict(dtest)
y_pred = (y_pred_prob >= 0.5).astype(int)


In [72]:
def predict_single(dtest, model):
    y_pred_prob = booster.predict(dtest)
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    return y_pred[0]

In [73]:
predict_single(dtest, booster)

1

In [75]:
import pickle 

with open('../models/churn-model.bin', 'wb') as f_out:
    pickle.dump(booster, f_out)

In [80]:
import requests
customer = X_test_customer.to_json()
url = 'http://localhost:9696/predict'
response = requests.post(url, json=customer)
result = response.json()
result

ConnectionError: HTTPConnectionPool(host='localhost', port=9696): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f1dda19a410>: Failed to establish a new connection: [Errno 111] Connection refused'))